https://neo4j.com/docs/neo4j-graphrag-python/current/user_guide_kg_builder.html

In [10]:
NEO4J_URI = "neo4j://localhost:7687"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "anatomy-exodus-pigment-colombo-octopus-1216"

In [1]:
import requests
from bs4 import BeautifulSoup

url = "https://www.lguplus.com/mobile/device/phone"

r = requests.get(url)

content = r.content.decode("utf-8")
content

'<!doctype html><html data-n-head-ssr lang="ko" data-n-head="%7B%22lang%22:%7B%22ssr%22:%22ko%22%7D%7D"><head ><title>휴대폰</title><meta data-n-head="ssr" name="viewport" content="width=device-width, initial-scale=0.48, maximum-scale=0.5, minimum-scale=0.3"><meta data-n-head="ssr" charset="utf-8"><meta data-n-head="ssr" http-equiv="X-UA-Compatible" content="IE=edge"><meta data-n-head="ssr" data-hid="description" name="description" content="휴대폰"><meta data-n-head="ssr" name="format-detection" content="telephone=no,address=no,email=no"><meta data-n-head="ssr" name="google-site-verification" content="FcOenpICu_iLLI1NF8NRwXPUsHOenpyScAfvcLS6_pM"><meta data-n-head="ssr" name="google-site-verification" content="-4XQrz2ukrgYIWKr31A0EDEpr4uIRVO2YvSYEjuMRXQ"><meta data-n-head="ssr" name="google-site-verification" content="QzpaVh18wRCxJmeKnLFdfJXYdNvXaSHWgqbqF_wKOQo"><meta data-n-head="ssr" name="naver-site-verification" content="756c5f0d06b8e657fba476b6ffa0de88efa8caa8"><meta data-n-head="ssr" na

In [35]:
# Neo4j Driver
import neo4j

neo4j_driver = neo4j.GraphDatabase.driver(
    NEO4J_URI,
    auth=(NEO4J_USERNAME, NEO4J_PASSWORD)
)

# LLM and Embedding Model
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.embeddings.openai import OpenAIEmbeddings

llm = OpenAILLM(
    model_name="gpt-4o",
    model_params={
        "response_format": {"type": "json_object"},  # use json_object formatting for best results
        "temperature": 0  # turning temperature down for more deterministic results
    }
)

# Graph Schema Setup
basic_node_labels = ["Object", "Entity", "Group", "Person", "Organization", "Place"]

academic_node_labels = ["ArticleOrPaper", "PublicationOrJournal"]

medical_node_labels = ["Anatomy", "BiologicalProcess", "Cell", "CellularComponent",
                       "CellType", "Condition", "Disease", "Drug",
                       "EffectOrPhenotype", "Exposure", "GeneOrProtein", "Molecule",
                       "MolecularFunction", "Pathway"]

node_labels = basic_node_labels + academic_node_labels + medical_node_labels

# define relationship types
rel_types = ["ACTIVATES", "AFFECTS", "ASSESSES", "ASSOCIATED_WITH", "AUTHORED", "BIOMARKER_FOR"]

#create text embedder
embedder = OpenAIEmbeddings()

# define prompt template
prompt_template = '''
Extract the entities (nodes) and specify their type from the following Input text.
Also extract the relationships between these nodes. the relationship direction goes from the start node to the end node.


Return result as JSON using the following format:
{{"nodes": [ {{"id": "0", "label": "the type of entity", "properties": {{"name": "name of entity" }} }}],
  "relationships": [{{"type": "TYPE_OF_RELATIONSHIP", "start_node_id": "0", "end_node_id": "1", "properties": {{"details": "Description of the relationship"}} }}] }}

...

Use only fhe following nodes and relationships:
{schema}

Assign a unique ID (string) to each node, and reuse it to define relationships.
Do respect the source and target node types for relationship and the relationship direction.

Do not return any additional information other than the JSON in it.

Examples:
{examples}

Input text:

{text}
'''

[#0000]  _: <POOL> created, routing address IPv4Address(('localhost', 7687))


In [36]:
import sys
import logging

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))

In [ ]:
# Knowledge Graph Builder
from neo4j_graphrag.experimental.components.text_splitters.fixed_size_splitter import FixedSizeSplitter
from neo4j_graphrag.experimental.pipeline.kg_builder import SimpleKGPipeline

kg_builder = SimpleKGPipeline(
    llm=llm,
    driver=neo4j_driver,
    text_splitter=FixedSizeSplitter(chunk_size=1024, chunk_overlap=100),
    embedder=embedder,
    # entities=node_labels,
    # relations=rel_types,
    prompt_template=prompt_template,
    from_pdf=False,
    on_error="RAISE"
)

result = await kg_builder.run_async(text=content)
print(f"Result: {result}")

PIPELINE_RUNNER: instantiating Pipeline from config type: PipelineType.SIMPLE_KG_PIPELINE
PIPELINE_RUNNER: instantiating Pipeline from config type: PipelineType.SIMPLE_KG_PIPELINE
PIPELINE_CONFIG: start parsing config...
PIPELINE_CONFIG: start parsing config...
PIPELINE_CONFIG: resolved 'extras': {'extras': {}}
PIPELINE_CONFIG: resolved 'extras': {'extras': {}}
PIPELINE_CONFIG: resolved globals: {'extras': {}, 'neo4j_config': {'default': <neo4j._sync.driver.Neo4jDriver object at 0x30eb5f4d0>}, 'llm_config': {'default': <neo4j_graphrag.llm.openai_llm.OpenAILLM object at 0x30eb7cc90>}, 'embedder_config': {'default': <neo4j_graphrag.embeddings.openai.OpenAIEmbeddings object at 0x30ec21e10>}}
PIPELINE_CONFIG: resolved globals: {'extras': {}, 'neo4j_config': {'default': <neo4j._sync.driver.Neo4jDriver object at 0x30eb5f4d0>}, 'llm_config': {'default': <neo4j_graphrag.llm.openai_llm.OpenAILLM object at 0x30eb7cc90>}, 'embedder_config': {'default': <neo4j_graphrag.embeddings.openai.OpenAIEmbe